In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import hddm
from sys import platform
import numpy as np
import seaborn as sns
import pandas as pd
from jupyterthemes import jtplot
import matplotlib.pyplot as plt
from kabuki.analyze import gelman_rubin



jtplot.style(theme='onedork')
jtplot.style(context='poster', fscale=2, spines=False, gridlines='--')
sns.set_color_codes("muted")

In [ ]:
if platform == 'linux2':
    home = '/home/krista/'
elif platform == 'darwin': 
    home = '/Users/Krista/'

In [ ]:
all_obs_data = hddm.load_csv(home + 'Dropbox/loki_0.5/analysis/aggregated_data/reward_ls_combined.csv')
all_obs_data = all_obs_data.rename(index=str, columns={"p_accuracy": "response",
                                                       "subj_id": "subj_idx"})
all_obs_data = all_obs_data[['response', 'rt', 'reward_code', 'subj_idx', 'ideal_B', 'cpp']] 
all_obs_data.reset_index(drop=True, inplace=True)

In [ ]:
all_obs_data.head()

In [ ]:
all_obs_data['cpp_shifted'] = all_obs_data.cpp.shift(1)
all_obs_data['ideal_B_shifted'] = all_obs_data.ideal_B.shift(1)

In [ ]:
all_obs_data.dropna(inplace=True) #need to drop nas for HDDMRegressor

In [ ]:
n_samples, n_burn = 3000, 500
n_effective_samples = n_samples - n_burn
subjects = all_obs_data.subj_idx.unique()

model_specifications = [('a~cpp_shifted', 'v~ideal_B_shifted'),
                        ('v~cpp_shifted', 'a~ideal_B_shifted'),
          ('a~1', 't~1', 'v~1')]
model_names = ['a_cpp_v_B', 'v_cpp_a_B', 'intercept']

model_inputs = dict(zip(model_names, model_specifications))
model_objects = dict()

In [ ]:
def estimate_regression_model(model_specification, model_name, subject, data, model_objects,
                              n_samples=n_samples, n_burn=n_burn, accuracy_coding=True, convergence_iteration=None):
    
    if accuracy_coding: 
        reg_model = hddm.HDDMRegressor(data = data, models = model_specification, bias=False, group_only_regressors=True, p_outlier=0.05) #accuracy coded
        reg_model.find_starting_values()
        reg_model.sample(n_samples, burn = n_burn, dbname = model_name, db='pickle')
    else: 
        reg_model = hddm.HDDMRegressor(data = data, models = model_specification, bias=True, group_only_regressors=True, p_outlier=0.05) #stim coded
        reg_model.find_starting_values()
        reg_model.sample(n_samples, burn=n_burn, dbname=model_name+'.db', db='pickle')
    
    if convergence_iteration is None: 
        model_objects[model_name + '_' + str(subject)] = reg_model
    else:
        model_objects[model_name + '_' + str(subject) + '_iter' + str(convergence_iteration)] = reg_model

In [ ]:
for subject in subjects:
    sub_data = all_obs_data.loc[all_obs_data.subj_idx == subject,]
    for model_name, model_specification in model_inputs.items(): 
        estimate_regression_model(model_specification, model_name, subject, sub_data, model_objects)
        print('model sampling complete')

In [ ]:
#run each model for each subject 5 times to estimate between-chain variance 
n_convergence_iterations = 5
model_convergence_objects = dict() 

for subject in subjects:
    sub_data = all_obs_data.loc[all_obs_data.subj_idx == subject,]
    for convergence_iteration in range(n_convergence_iterations):
        for model_name, model_specification in model_inputs.items(): 
            estimate_regression_model(model_specification=model_specification, model_name = model_name, subject=subject, 
                                      data=sub_data, model_objects=model_convergence_objects, convergence_iteration=convergence_iteration)

In [ ]:
n_models = len(model_names)
print(n_models)
len(model_convergence_objects) == (len(subjects) * n_convergence_iterations * n_models)

In [ ]:
var_criterion = 1.1 #max. acceptable ratio of between to within chain variance (see Gelman)
subjects = all_obs_data.subj_idx.unique().astype('str')
search_strings = [model_name + '_' +  subject for model in model_names for subject in subjects]

In [ ]:
unconverged = list()

for string in search_strings:
        data = dict((key, value) for key, value in model_convergence_objects.iteritems() if key.startswith(string))
        R_dict = gelman_rubin(data.values()) #create dict of R hat statistics for each parameter  
        print(R_dict)
        unconverged.append(any(param > var_criterion for param in R_dict.itervalues())) #check whether stat. exceeds criterion 

In [ ]:
n_unconverged_params = sum(unconverged) #find number of unconverged parameters 
print('all parameters have converged:' , n_unconverged_params == 0)

In [ ]:
locals().update(model_objects) # convert keys and values to variables 

In [ ]:
len(subj_df_list) == (len(subjects) * n_models)

In [ ]:
len(subj_df_all) == (len(subjects)*n_models*n_effective_samples)

In [ ]:
a_cpp_v_B = [a_cpp_v_B_786, a_cpp_v_B_787, a_cpp_v_B_788, a_cpp_v_B_789]
v_cpp_a_B = [v_cpp_a_B_786, v_cpp_a_B_787, v_cpp_a_B_788, v_cpp_a_B_789]

In [ ]:
intercept = [intercept_786, intercept_787, intercept_788, intercept_789]

In [ ]:
all_models = [a_cpp_v_B, v_cpp_a_B, intercept]
print(model_names)

In [ ]:
model_dict = dict(zip(model_names, all_models))

In [ ]:
subj_df_list = []

for model in model_dict:
    for subject in range(len(subjects)):
        subj_df = model_dict[model][subject].get_traces()
        subj_df['subj_idx'] = model_dict[model][subject].data.subj_idx.unique().tolist() * n_effective_samples
        subj_df['model'] = model
        subj_df['dic'] = model_dict[model][subject].dic
        subj_df_list.append(subj_df)
    
subj_df_all = pd.concat(subj_df_list, sort=True)

In [ ]:
subj_df_all.head()

In [ ]:
subj_df_all.model.unique()

In [ ]:
subj_df_all.groupby(['subj_idx', 'model'])['dic'].unique()

In [ ]:
dics_df = (subj_df_all.groupby(['subj_idx', 'model'])['dic'].unique() - subj_df_all.loc[subj_df_all.model == 'intercept'].groupby(['subj_idx'])['dic'].unique()).reset_index()

In [ ]:
dics_df['raw_dic'] = subj_df_all.groupby(['subj_idx', 'model'])['dic'].unique().reset_index()['dic']

In [ ]:
dics_df = dics_df.rename(index=str, columns={"dic": "null_adj_dic"})
dics_df['null_adj_dic'] = dics_df['null_adj_dic'].str[0]
dics_df['raw_dic'] = dics_df['raw_dic'].str[0]

In [ ]:
dics_df.head()

In [ ]:
subj_df_all.to_csv(home + 'Dropbox/loki_0.5/analysis/aggregated_data/subjectwise_ddm_reg.csv')

In [ ]:
dics_df.to_csv(home + 'Dropbox/loki_0.5/analysis/aggregated_data/subjectwise_dics_reg.csv' )